In [408]:
from typing import List
from datetime import date

In [409]:
class Groupe:
    def __init__(self, annee: int, ecole: str, identifiant: str, nom_professeur: str):
        print("Instanciation du groupe")
        self.annee = annee
        self.ecole = ecole
        self.identifiant = identifiant
        self.nom_professeur = nom_professeur
        self.eleves: List["Eleve"] = []

    def __str__(self) -> str:
        return f"Le groupe a pour professeur {self.nom_professeur}"

    def calculer_moyenne(self) -> float:
        """Fonction permettant de calculer la moyenne
        d'un groupe

        Raises:
            ZeroDivisionError: Si la liste d'élèves est vide
            alors une erreur est provoquée

        Returns:
            float: Valeur de la moyenne du groupe
        """
        # sum(self.liste_eleves) => ne donne pas l'information recherchée. Sum effectue la somme des éléments d'une liste
        # or les éléments de la liste liste_eleves sont des objets de type Eleve et aucune méthode __sum__ n'est définie sur cette
        # classe. Plutôt prévilégier la longueur de la liste
        if len(self.eleves) == 0:
            raise ZeroDivisionError("Le groupe ne contient pas d'élèves")

        notes_groupe = [note for eleve in self.eleves for note in eleve.notes.values()]
        if not notes_groupe:
            raise ZeroDivisionError("Le groupe d'élèves ne contient pas de notes")

        moyenne_groupe, eleves_notes = 0, 0
        for eleve in self.eleves:
            try:
                moyenne_groupe += eleve.calculer_moyenne()
                eleves_notes += 1
            except ValueError:
                continue
        return moyenne_groupe / eleves_notes

    # COMPARAISONS
    def __gt__(self, other: "Groupe") -> bool:
        return self.calculer_moyenne() > other.calculer_moyenne()

    def __lt__(self, other: "Groupe") -> bool:
        return self.calculer_moyenne() < other.calculer_moyenne()

    def __ge__(self, other: "Groupe") -> bool:
        return self.calculer_moyenne() >= other.calculer_moyenne()

    def __le__(self, other: "Groupe") -> bool:
        return self.calculer_moyenne() <= other.calculer_moyenne()

    def __eq__(self, other: "Groupe") -> bool:
        return self.calculer_moyenne() == other.calculer_moyenne()

    def __ne__(self, other: "Groupe") -> bool:
        return self.calculer_moyenne() != other.calculer_moyenne()

In [410]:
class Examen:
    def __init__(self, matiere: str, date_examen: date, nom: str, groupe: Groupe):
        print("Instanciation de l'examen")
        self.matiere = matiere
        self.date_examen = date_examen
        self.nom = nom
        self.groupe = groupe

    def __str__(self) -> str:
        return f"L'examen a lieu le {self.date_examen}"

    def calculer_moyenne(self) -> float:
        """Permet de calculer la moyenne d'un examen

        Raises:
            ValueError: Provoque une erreur s'il n'existe pas de note

        Returns:
            float: La moyenne du groupe à l'examen
        """
        notes_examen = []
        for eleve in self.groupe.eleves:
            notes_examen.extend(
                note for examen, note in eleve.notes.items() if self == examen
            )
        if not notes_examen:
            raise ValueError("Il n'y a pas de note")
        return sum(notes_examen) / len(notes_examen)

In [411]:
class Eleve:
    """
    Ceci est la doc de la classe Eleve
    """

    def __init__(
        self,
        nom: str,
        prenom: str,
        date_naissance: date,
        civilité: str,
        groupe: Groupe,
        redoublant: bool = False,
    ):
        print("Instanciation de l'élève")
        self.nom = nom
        self.prenom = prenom
        self.date_naissance = date_naissance
        self.civilite = civilité
        self.redoublant = redoublant
        self.groupe = groupe
        self.notes: dict[
            Examen, int
        ] = {}  # mettre des informations de typing même pour les dicts

    def __str__(self) -> str:  # affiche nom et prénom
        return f"Le nom de l'élève est {self.prenom} {self.nom}"

    def calculer_moyenne(self) -> float:
        """Calcule la moyenne d'un élève à partir
        de toutes ses notes

        Returns:
            float: Moyenne de l'élève
        """
        # on regarde si le dictionnaire est vide et donc ne contient pas de note
        if len(self.notes.values()) == 0:
            raise ValueError(
                "La moyenne ne devrait pas exister avec une liste sans notes"
            )
        return sum(self.notes.values()) / len(self.notes.values())

    def ajouter_note(self, examen: Examen, note: float) -> None:
        """Ajoute un examen avec une note dans les notes de l'élève
        seulement si l'examen n'est pas déjà présent

        Args:
            examen (Examen): examen à ajouter
            note (float): valeur de la note obtenue à l'examen
        """
        if examen in self.notes.keys():
            raise KeyError("L'examen existe déjà")
        if not (0 <= note <= 20):
            raise ValueError("La note n'est pas comprise entre 0 et 20 inclus")
        self.notes[examen] = note

In [412]:
from datetime import date
import statistics


def test_project():  # sourcery skip: use-contextlib-suppress
    g = Groupe(2022, "ENSAI", "datascience#1", "Gaëlle")
    exams = {
        "ex1": Examen("dev", date(2022, 9, 29), "Premier projet Python", g),
        "ex2": Examen("dev", date(2022, 10, 10), "Second projet Python", g),
        "ex3": Examen("math", date(2022, 10, 3), "Rappel sur l'arithmétique", g),
    }
    try:
        g.calculer_moyenne()
        raise AssertionError("La moyenne ne devrait pas exister sur un groupe vide")
    except (ValueError, ZeroDivisionError, statistics.StatisticsError):
        pass
    eleves = {
        "alice": Eleve("Test", "Alice", date(1970, 1, 1), "Mme", g),
        "bob": Eleve("Démo", "Bob", date(2000, 1, 1), "M", g),
        "charlie": Eleve("Test", "Charlie", date(1999, 4, 3), "Mx", g),
        "alice2": Eleve("Doublon", "Alice", date(2022, 1, 1), "Mme", g),
    }
    g.eleves = list(eleves.values())

    try:
        # ici
        g.calculer_moyenne()
        raise AssertionError(
            "La moyenne ne devrait pas  exister sur un groupe sans notes"
        )
    except (ValueError, ZeroDivisionError, statistics.StatisticsError):
        pass
    for e in g.eleves:
        try:
            e.calculer_moyenne()
            raise AssertionError("La moyenne ne devrait pas  exister sur un élève vide")
        except (ValueError, ZeroDivisionError, statistics.StatisticsError):
            pass

    eleves["alice"].ajouter_note(exams["ex1"], 18)
    eleves["alice"].ajouter_note(exams["ex2"], 14)
    eleves["alice"].ajouter_note(exams["ex3"], 12)
    try:
        eleves["alice"].ajouter_note(exams["ex1"], 17)
        raise AssertionError("L'examen ne peut pas être noté plusieurs fois")
    except KeyError:
        pass
    try:
        eleves["bob"].ajouter_note(exams["ex1"], 29)
        raise AssertionError("La note 29 devrait être interdite")
    except ValueError:
        pass
    try:
        eleves["bob"].ajouter_note(exams["ex1"], -1)
        raise AssertionError("La note -1 devrait être interdite")
    except ValueError:
        pass
    eleves["bob"].ajouter_note(exams["ex1"], 0)
    eleves["bob"].ajouter_note(exams["ex2"], 20)
    eleves["alice2"].ajouter_note(exams["ex1"], 13)
    assert (
        round(eleves["alice"].calculer_moyenne(), 2) == 14.67
    ), "La moyenne d'Alice Test est incorrecte"
    assert (
        round(eleves["bob"].calculer_moyenne()) == 10
    ), "La moyenne de Bob Démo est incorrecte"
    try:
        eleves["charlie"].calculer_moyenne()
        raise AssertionError("Charlie Test ne devrait pas avoir de moyenne")
    except (ValueError, ZeroDivisionError, statistics.StatisticsError):
        pass
    assert (
        eleves["alice2"].calculer_moyenne() == 13
    ), "La moyenne d'Alice Doublon est incorrecte"

    assert round(g.calculer_moyenne(), 2) == 12.56
    assert round(exams["ex1"].calculer_moyenne(), 2) == 10.33
    # assert exams["ex2"].calculer_moyenne() == 20 # erreur à priori car bob et alice ont une note
    assert round(exams["ex2"].calculer_moyenne(), 2) == round((20 + 14) / 2)
    assert exams["ex3"].calculer_moyenne() == 12
    # try:
    #     # exams["ex3"].calculer_moyenne() # il existe une note dans l'examen 3
    #     raise AssertionError("L'examen 3 ne devrait pas avoir de moyenne")
    # except (ValueError, ZeroDivisionError, statistics.StatisticsError):
    #         pass

In [413]:
import sys

try:
    test_project()
    print("Bravo, les tests passent !")
except AssertionError as e:
    print(f"Erreur de logique : {e}", file=sys.stderr)
except Exception as e:
    print(f"Erreur Python : {e}", file=sys.stderr)

Instanciation du groupe
Instanciation de l'examen
Instanciation de l'examen
Instanciation de l'examen
Instanciation de l'élève
Instanciation de l'élève
Instanciation de l'élève
Instanciation de l'élève
Bravo, les tests passent !
